In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor, nn

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## NN module
The <mark>`NN` module contains classes, functions, and other modules for creating neural networks from smaller building blocks</mark>. The majority of classes inherit from `nn.Module`, which provides a lot of functionality for tracking learnable parameters and acting on inputted tensors.

A class inheriting from `nn.Module` will have a:
- `forward()` method, where it can act on incoming tensors. The `__call__` method will call the `forward` method, meaning that once instantiated, the object can be called, in order to call it's `forward` method, e.g. `my_module(x)` will pass `x` to `my_module.forward(x)` and return the output.
- `parameters()` method, which is a provides a recursive generator that yields all `nn.Parameter`s stored by the `Module` and any other `nn.Module`s it stores
- `state_dict()` method, which returns a dictionary of the current values of all `nn.Parameter`s and registered buffers stored by the `Module` and any other `nn.Module`s it stores
- `to()` method, which will recursively place all other `nn.Module`s and `nn.Parameter`s stored by the `Module` onto the specified device


### nn.Parameter
<mark>`nn.Parameter`s are basically just `Tensor`s with `require_grad=True`</mark>, except that when they are declared as attributes of an `nn.Module`, they will be treated specially. E.g. they are returned by the `parameters()` generator, and stored in the `state_dict`. As we'll see later, <mark>optimisers in PyTorch are initialised using the `parameters()` generator, so `nn.Parameter`s will therefore be updated by gradient descent. Additionally, loading and saving of a `nn.Module` is done via its `state_dict`, so the values of `nn.Parameter`s will be loaded and saved, too.</mark>

In [3]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()  # The super constructor must always be called, otherwise no parameters can be assigned
        self.tensor_a = torch.tensor([3.], requires_grad=True)  # here we declare a tensor with gradient
        self.param_b = nn.Parameter(torch.tensor([2.]))  # here we declare a parameter with gradient

In [4]:
module = MyModule()

In [5]:
list(module.parameters())  # note that only param_b is listed, tensor_a is ignored

[Parameter containing:
 tensor([2.], requires_grad=True)]

In [7]:
module.state_dict()  # similarly, only param_b is listed

OrderedDict([('param_b', tensor([2.]))])

<mark>Parameters can also be included as `nn.ParameterList` and `nn.ParameterDict` classes, which act similarly to lists and dictionaries, except that they will also be identified as parameters of the module.</mark>

### Buffers
Sometimes we have <mark>values that we want to keep constant during optimisation, but also want to be included in the `state_dict` such that they can be easily loaded and saved. Such values can be registered as *buffers*:</mark>

In [8]:
class MyModule(nn.Module):
    def __init__(self, value):
        super().__init__()
        self.tensor_a = torch.tensor([3.], requires_grad=True)
        self.param_b = nn.Parameter(torch.tensor([2.]))
        self.register_buffer('buffer_c', value)  # register the buffer with a given name

In [9]:
module = MyModule(Tensor([-1]))

In [11]:
list(module.parameters())  # buffer_c isn't included as a parameter, tensor_a is again ignored

[Parameter containing:
 tensor([2.], requires_grad=True)]

In [12]:
module.state_dict()  # but is included in the state dict specifying also its value, tensor_a is again ignored

OrderedDict([('param_b', tensor([2.])), ('buffer_c', tensor([-1.]))])

In [13]:
module.buffer_c  # the buffer appears as an attribute with the name that was provided when it was registered

tensor([-1.])

## Common classes
There are many different classes implemented in PyTorch. See https://pytorch.org/docs/stable/nn.html for the full list. Described below are a <mark>few common examples.</mark>

### Linear layers

A common class is `nn.Linear`, which implements the linear transform `w.x+b`, where `w` and `b` are learnable parameters. These can be used for the "hidden" layers in feed-forward DNNs

In [14]:
lin = nn.Linear(in_features=4, out_features=6)  # the layer expects 4 features in and will output 6 features

In [15]:
lin.state_dict()  # it has a weight (6,4) and a bias (6), which are initialised at random

OrderedDict([('weight',
              tensor([[-0.1635,  0.1237, -0.4205,  0.2092],
                      [-0.1941,  0.4541, -0.3560,  0.4676],
                      [-0.4014, -0.1734, -0.0160,  0.1873],
                      [ 0.1532, -0.2695,  0.0614,  0.0183],
                      [ 0.3869,  0.1485, -0.0024,  0.0363],
                      [-0.1718,  0.4823, -0.1703,  0.4349]])),
             ('bias',
              tensor([-0.3623, -0.2044,  0.4976, -0.2128,  0.4963,  0.4201]))])

In [16]:
x = torch.randn(10,4)
x = lin(x)  # this calls the forward method of the linear layer, which applies the linear transformation to the incoming x tensor
x.shape, x.grad_fn  # note that the linear transform was broadcast across the first dimension of x, and that x now has a grad function

(torch.Size([10, 6]), <AddmmBackward0 at 0x1179c1db0>)

### Activation layers
<mark>Sometimes, the classes don't have any learnable parameters, but it is more convenient to treat them as `nn.Module`s. Activation functions are typical examples:</mark>

In [17]:
act = nn.ReLU()
act.state_dict()

OrderedDict()

In [18]:
act(x)

tensor([[0.6493, 0.9782, 0.9184, 0.0000, 0.3097, 1.2477],
        [0.0000, 0.0000, 0.2852, 0.0000, 0.7568, 0.2353],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7528, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0073, 0.8428, 0.0000],
        [0.0000, 0.0641, 0.8788, 0.0000, 0.0717, 0.6831],
        [0.0000, 0.2859, 0.6791, 0.0000, 0.3476, 0.8128],
        [0.0980, 0.2057, 0.3449, 0.0000, 0.6776, 0.6156],
        [0.0000, 0.3588, 0.4100, 0.0000, 0.8765, 1.1306],
        [0.0000, 0.7563, 0.2249, 0.0000, 0.6159, 1.4330],
        [0.0000, 0.0000, 0.1193, 0.0000, 0.4123, 0.5194]],
       grad_fn=<ReluBackward0>)

See https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity for the full list of activation functions implemented in PyTorch

### Sequential
Above, we took some data and passed it through a linear layer and then through an activation function. This is a very common action in a neural network. <mark>Sometimes it can be convenient to group together layers and modules into an `nn.Sequential` class, which takes multiple `nn.Module`s and when its `forward` method is called, it will feed the input to the first module and then sequentially feed the output into the next module, and so on, finally returning the output of the last module</mark>

In [19]:
lin_act = nn.Sequential(lin, act)

In [20]:
lin_act(torch.randn(10,4))

tensor([[0.0000e+00, 0.0000e+00, 7.5461e-01, 2.1361e-01, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.2574e-03, 5.5305e-01, 0.0000e+00, 3.2269e-01, 8.7673e-01],
        [0.0000e+00, 0.0000e+00, 3.5844e-01, 0.0000e+00, 6.5265e-01, 4.2333e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.4023e-04, 1.2679e+00, 4.3317e-01],
        [0.0000e+00, 8.6462e-01, 0.0000e+00, 0.0000e+00, 1.3742e+00, 1.5711e+00],
        [0.0000e+00, 2.3805e-01, 1.1370e+00, 0.0000e+00, 0.0000e+00, 1.0657e+00],
        [3.4723e-01, 6.8077e-01, 1.0316e+00, 0.0000e+00, 4.1028e-02, 1.1070e+00],
        [3.1998e-01, 1.1306e+00, 8.9459e-01, 0.0000e+00, 2.9035e-01, 1.6655e+00],
        [0.0000e+00, 0.0000e+00, 6.2697e-01, 7.6244e-02, 4.2316e-01, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0623e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       grad_fn=<ReluBackward0>)

In [21]:
lin_act.state_dict()  # the parameters of the linear layer are still contained in the state_dict o the sequential module

OrderedDict([('0.weight',
              tensor([[-0.1635,  0.1237, -0.4205,  0.2092],
                      [-0.1941,  0.4541, -0.3560,  0.4676],
                      [-0.4014, -0.1734, -0.0160,  0.1873],
                      [ 0.1532, -0.2695,  0.0614,  0.0183],
                      [ 0.3869,  0.1485, -0.0024,  0.0363],
                      [-0.1718,  0.4823, -0.1703,  0.4349]])),
             ('0.bias',
              tensor([-0.3623, -0.2044,  0.4976, -0.2128,  0.4963,  0.4201]))])

### Module lists and dicts
Similar to `nn.ParameterList` and `nn.ParameterDict`, <mark>`nn.ModuleList` and `nn.ModuleDict` can be used to contain multiple modules and have them be recognised by the parent `nn.Module` as modules:</mark>

In [22]:
mlist= nn.ModuleList([lin, act])

In [23]:
isinstance(mlist, nn.Module)

True

In [24]:
mlist(x)  # does not act like a Sequential

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [25]:
x = torch.rand(10,4)
for m in mlist: x = m(x)  # but can be iterated through
x

tensor([[0.0000, 0.0000, 0.3315, 0.0000, 0.7462, 0.6536],
        [0.0000, 0.0000, 0.1725, 0.0000, 0.8319, 0.4915],
        [0.0000, 0.0707, 0.5480, 0.0000, 0.5823, 0.7847],
        [0.0000, 0.0000, 0.1048, 0.0000, 0.8789, 0.3783],
        [0.0000, 0.0255, 0.2686, 0.0000, 0.7424, 0.6854],
        [0.0000, 0.0000, 0.2530, 0.0000, 0.8254, 0.4819],
        [0.0000, 0.0000, 0.1299, 0.0000, 0.8457, 0.5606],
        [0.0000, 0.0000, 0.0691, 0.0000, 0.9400, 0.5128],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.9867, 0.6592],
        [0.0000, 0.0000, 0.3557, 0.0000, 0.7471, 0.5475]],
       grad_fn=<ReluBackward0>)

## Fowards pass
<mark>When building a new module, it is necessary to implement the forwards pass, which defines how the parameters and child modules will affect the incoming tensors. When dealing with high-level PyTorch, the backwards pass will be automatically implemented.</mark>

In [26]:
class LinAct(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.lin = nn.Linear(nin, nout)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.lin(x)
        x = self.act(x)
        return x

In [27]:
lin_act = LinAct(2,4)

In [26]:
x = torch.randn(5,2)
lin_act(x)

tensor([[0.7515, 0.9448, 0.4860, 0.0000],
        [0.7264, 0.0000, 0.3230, 0.0000],
        [0.0744, 0.1151, 0.0000, 0.0000],
        [0.9014, 0.0000, 0.5478, 0.0000],
        [0.2433, 1.8337, 0.0000, 0.8897]], grad_fn=<ReluBackward0>)

In [28]:
lin_act.state_dict()

OrderedDict([('lin.weight',
              tensor([[ 0.6610, -0.1656],
                      [-0.5771,  0.5966],
                      [ 0.5921,  0.3726],
                      [-0.2306,  0.1094]])),
             ('lin.bias', tensor([-0.2290,  0.3584,  0.3549,  0.5362]))])

<mark>In the forward method, nothing is compiled, meaning that conditionals can be used to change what actions are applied</mark>, depending on the data provided at runtime:

In [29]:
class LinAct(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.lin = nn.Linear(nin, nout)
        self.act = nn.ReLU()
        self.zero_replacement = nn.Parameter(Tensor([-3]))
        
    def forward(self, x):
        x = self.lin(x)
        x = self.act(x)
        x[x<=0] = self.zero_replacement  # if any values are less than or equal to 0, replace them with a learnable default value
        return x

In [30]:
lin_act = LinAct(2,4)

In [31]:
x = torch.randn(5,2)
lin_act(x)

tensor([[ 1.2585, -3.0000,  0.4568,  0.7101],
        [-3.0000, -3.0000, -3.0000, -3.0000],
        [ 1.2667, -3.0000, -3.0000,  0.8093],
        [-3.0000, -3.0000,  0.5913, -3.0000],
        [-3.0000, -3.0000,  0.1393, -3.0000]], grad_fn=<IndexPutBackward0>)

## Initialisation
<mark>Suitable initialisation of parameters in a neural network is key to making them trainable quickly (or at all)</mark>. PyTorch provides a default init scheme, but this isn't guaranteed to be suitable for the networks you create; <mark>it should vary according to, at least, the activation function used</mark>. The general <mark>rule-of-thumb is that data sampled from a unit-Gaussian, should retain a unit-Gaussian shape when passed through the DNN. To see why this is important, check out this interactive demo https://www.deeplearning.ai/ai-notes/initialization/index.html.</mark>

Two of the most common schemes are:
- <mark>Xavier Glorot: weights are sampled from either a uniform distribution between ±sqrt(6/(nin+nout)), or a Gaussian with mean 0 and std sqrt(2/(nin+nout)). This is used for e.g. linear, sigmoid, softmax, and tanh activation functions.</mark>
- <mark>Kaiming He: weights are sampled from either a uniform distribution between ±sqrt(3/nin), or a Gaussian with mean 0 and std sqrt(1/nin). This is used for e.g. ReLU, PReLU, Swish, and Mish activation functions.</mark>

As part of my LUMIN package, I maintain a list of applicable init schemes here https://github.com/GilesStrong/lumin/blob/master/lumin/nn/models/initialisations.py

The <mark>bias of a linear layer can generally be initialised to zeros</mark>

A full list of the init schemes in PyTorch is found here https://pytorch.org/docs/stable/nn.init.html

In [34]:
lin = nn.Linear(3,5)
lin.state_dict()

OrderedDict([('weight',
              tensor([[-0.4279, -0.5738,  0.0695],
                      [-0.1682,  0.3995, -0.3439],
                      [-0.4387, -0.4449,  0.4781],
                      [-0.4473,  0.2167, -0.5135],
                      [ 0.5461,  0.3604,  0.0592]])),
             ('bias', tensor([-0.2564,  0.5221,  0.1372, -0.2761,  0.2770]))])

<mark>This will set initial values in-place (note the _ at the end of the method to indicate its an in-place operation)</mark>. Since <mark>we expect to feed the output into a ReLU, we need to specify 'relu' for the nonlinearity</mark>. For no discernably good reason at all, there is a <mark>default value of 'leaky_relu' so don't forget to correct this.</mark>

In [35]:
nn.init.kaiming_normal_(lin.weight, nonlinearity='relu')

Parameter containing:
tensor([[-0.1404, -0.0241,  0.7050],
        [-0.0108,  0.2396, -0.4940],
        [-0.1710,  0.1022,  0.8184],
        [-0.1666,  0.2049, -0.3602],
        [ 0.5765, -0.5232,  0.5404]], requires_grad=True)

Let's zero the bias

In [36]:
nn.init.zeros_(lin.bias)

Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)

In [37]:
lin.state_dict()

OrderedDict([('weight',
              tensor([[-0.1404, -0.0241,  0.7050],
                      [-0.0108,  0.2396, -0.4940],
                      [-0.1710,  0.1022,  0.8184],
                      [-0.1666,  0.2049, -0.3602],
                      [ 0.5765, -0.5232,  0.5404]])),
             ('bias', tensor([0., 0., 0., 0., 0.]))])

<mark>When writing a new module, generally I make sure that the layers are correctly initialised just after they are declared</mark>. However, it is still possible to reinitialise an instantiated module by recursively searching through it for different layers, like:

In [38]:
def init_net(model:nn.Module):
    r'''Recursively initialise fully-connected ReLU network with Kaiming and zero bias'''
    if isinstance(model,nn.Linear):
        init.kaiming_normal_(model.weight, nonlinearity='relu')
        init.zeros_(model.bias)
    for l in model.children(): init_net(l)

However you now lose a bit of control, and it's easy to use the "wrong" init scheme depending on the layer.

## Saving/loading
After training a neural network (or even during), we often want to <mark>save its parameters. This is done via the `state_dict`.</mark>

In [39]:
state = lin_act.state_dict()
state

OrderedDict([('zero_replacement', tensor([-3.])),
             ('lin.weight',
              tensor([[ 0.6140,  0.6540],
                      [ 0.5110, -0.1025],
                      [-0.2431,  0.2308],
                      [ 0.3604,  0.2854]])),
             ('lin.bias', tensor([-0.1432, -0.6213,  0.0942,  0.0719]))])

In [40]:
torch.save(state, '03_save.pt')

Now later we may want to reload the saved parameters. Note that <mark>we have only saved the parameters, and not the class or code itself</mark>. So we would need to <mark>re-instantiate the network with random parameters, and then load the saved params into it:</mark>

In [41]:
new_lin_act = LinAct(2,4)
new_lin_act.state_dict()

OrderedDict([('zero_replacement', tensor([-3.])),
             ('lin.weight',
              tensor([[ 0.0491,  0.6193],
                      [-0.7037,  0.1437],
                      [ 0.4089,  0.4134],
                      [ 0.5533,  0.1788]])),
             ('lin.bias', tensor([-0.6660,  0.4402, -0.1380, -0.2449]))])

In [42]:
loaded_state = torch.load('03_save.pt')

In [43]:
new_lin_act.load_state_dict(loaded_state)

<All keys matched successfully>

In [44]:
new_lin_act.state_dict()

OrderedDict([('zero_replacement', tensor([-3.])),
             ('lin.weight',
              tensor([[ 0.6140,  0.6540],
                      [ 0.5110, -0.1025],
                      [-0.2431,  0.2308],
                      [ 0.3604,  0.2854]])),
             ('lin.bias', tensor([-0.1432, -0.6213,  0.0942,  0.0719]))])

A more tricky, but flexible and user-convenient export method is the <mark>PyTorch packager https://pytorch.org/docs/stable/package.html. This allows one to export the saved parameters, the code necessary to produce the modules it relates to, and relevant code from any dependencies that users may not want to install.</mark>

## nn.functional
Many operations/layers are presented as classes, but <mark>most are also available as functions; https://pytorch.org/docs/stable/nn.functional.html -- commonly imported as `F`.</mark> Without moving to a full functional-programming approach it is still possible/useful to use these functions in you modules, e.g.:

In [45]:
import torch.nn.functional as F

In [46]:
class LinAct(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.lin = nn.Linear(nin, nout)
        
    def forward(self, x):
        x = self.lin(x)
        x = F.relu(x)  # Note rather than have the ReLU be an object of the module, we use the function version
        return x

In [47]:
x = torch.randn(4,2)
x = LinAct(2,6)(x)
x

tensor([[0.0000, 0.0000, 0.6229, 0.0000, 0.2312, 0.6109],
        [0.0000, 0.0000, 0.4761, 0.0000, 0.2108, 0.1984],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5197, 0.9931],
        [0.0000, 0.0000, 0.3721, 0.1287, 0.1985, 0.0000]],
       grad_fn=<ReluBackward0>)

<mark>A fully functional approach would be something like:</mark>

In [49]:
w = nn.init.kaiming_normal_(torch.empty(6,2))
b = torch.zeros(6)

In [50]:
x = torch.randn(4,2)
F.relu(F.linear(x, weight=w, bias=b))

tensor([[0.6192, 1.7173, 0.0000, 0.7382, 0.7151, 0.0000],
        [0.0000, 2.6393, 0.4481, 7.6317, 0.0000, 0.0000],
        [0.9490, 4.2962, 0.0000, 4.8934, 0.7841, 0.0000],
        [1.0552, 0.8990, 0.0000, 0.0000, 1.5985, 0.0000]])

Or even:

In [51]:
torch.clamp_min(x@w.T+b, 0)

tensor([[0.6192, 1.7173, 0.0000, 0.7382, 0.7151, 0.0000],
        [0.0000, 2.6393, 0.4481, 7.6317, 0.0000, 0.0000],
        [0.9490, 4.2962, 0.0000, 4.8934, 0.7841, 0.0000],
        [1.0552, 0.8990, 0.0000, 0.0000, 1.5985, 0.0000]])